# 1. 确认GPU

In [1]:
!nvidia-smi

Fri Sep  1 08:28:10 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.76       Driver Version: 515.76       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:8A:00.0 Off |                  N/A |
| 77%   74C    P2   302W / 350W |   6285MiB / 24576MiB |     96%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:8B:00.0 Off |                  N/A |
| 87%   

In [9]:
import torch
from torch import nn

torch.device('cpu'), torch.cuda.device('cuda'), torch.cuda.device('cuda:0')

(device(type='cpu'),
 <torch.cuda.device at 0x7f11605be9d0>)

In [3]:
print(torch.cuda.device_count())

4


In [10]:
def try_gpu(i=0):
    """如果存在，则返回gpu(i)，否则返回gpu"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

def try_all_gpus():
    """返回所有可用的GPU，如果没有GPU，则返回[cpu(),]。"""
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]      
    return devices if devices else [torch.device('cpu')]
                    
print(try_gpu())
print(try_gpu(10))
print(try_all_gpus())

cuda:0
cpu
[device(type='cuda', index=0), device(type='cuda', index=1), device(type='cuda', index=2), device(type='cuda', index=3)]


In [11]:
# 查询张量所在设备
X = torch.tensor([1,2,3])
print(X.device) # 默认在CPU内存上

cpu


In [12]:
# 存储在GPU上
X = torch.ones(2,3,device=try_gpu())
X 

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [13]:
# 在第二个GPU上创建一个随即张量
Y = torch.rand(2,3,device=try_gpu(1))
print(Y.device) # 没有1号GPU，则放到CPU上
Y

cuda:1


tensor([[0.6188, 0.0115, 0.6101],
        [0.3390, 0.1095, 0.1298]], device='cuda:1')

In [18]:
# 要计算X+Y，我们需要决定在哪里执行这个操作
Z = X.cuda(0) # X+Y必须X和Y都在同一个GPU上
print(X)
print(Z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [21]:
Y = torch.rand(2,3,device=try_gpu(0))
print(Y + Z)
Z.cuda(0) is Z # 如果变量在0号GPU时，就返回True

tensor([[1.8523, 1.3706, 1.9944],
        [1.3185, 1.9515, 1.8636]], device='cuda:0')


True

In [22]:
# 神经网络与GPU
net = nn.Sequential(nn.Linear(3,1)) # 创建神经网络时已经把权重初始化好了
net = net.to(device=try_gpu()) # 把所有参数在0号GPU上拷贝一份
X = torch.ones(2,3,device=try_gpu()) # X 在0号GPU上
net(X) # 所以前项运算所有元素都在0号GPU上运行

tensor([[0.9475],
        [0.9475]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [23]:
# 确认模型参数存储在同一个GPU上
net[0].weight.data.device

device(type='cuda', index=0)